In [ ]:
import sys
sys.path.append('../python')
from get_vs import locate_vs
from misc import *

import pandas as pd
from pandas import DataFrame
import numpy as np
import os
from tqdm import tqdm
import xarray as xr
import gcsfs
from dask_kubernetes import KubeCluster
from dask.distributed import Client

%matplotlib inline

In [ ]:
is_pangeo_data = True # True if in Pangeo binder, False if in laptop

In [ ]:
if is_pangeo_data:
    cluster = KubeCluster(n_workers=10)
    client = Client(cluster)

The coordinates of the virtual stations in [Hydroweb](http://hydroweb.theia-land.fr) don't match with the rivers in [HydroSHEDS](http://www.hydrosheds.org). In order to find the corresponding coordinates in HydroSHEDS, we look around the original position for the pixel with the biggest accumulated flow which is bigger than a minimum flow. If no such flow is found, we look further around, until we find one (but not too far away, in which case we just drop the virtual station). The new_lat/new_lon are the coordinates of this pixel, if found.

In [ ]:
if not os.path.exists('../data/amazonas/amazonas.pkl'):
    df = locate_vs('../data/amazonas/amazonas.txt', pix_nb=20, acc_min=1_000_000)
    df.to_pickle('../data/amazonas/amazonas.pkl')
else:
    df = pd.read_pickle('../data/amazonas/amazonas.pkl')

In [ ]:
sub_latlon = df[['new_lat', 'new_lon']].dropna().values
print(f'Out of {len(df)} virtual stations in Hydroweb, {len(sub_latlon)} could be found in HydroSHEDS.')

The following coordinates are duplicated because some virtual stations fall inside the same pixels.

In [ ]:
rm_latlon = [(-4.928333333333334, -62.733333333333334), (-3.8666666666666667, -61.6775)]

In [ ]:
df_ll = df[['new_lat', 'new_lon']].dropna()
duplicated = df_ll[df_ll.duplicated(keep=False)]
duplicated

In [ ]:
assert len(duplicated) / 2 == len(rm_latlon)

In [ ]:
# all the subbasins in the hydrologic partition (including virtual stations)
if is_pangeo_data:
    gcs_w_token = gcsfs.GCSFileSystem(project='pangeo-data', token='browser')
    labels = [os.path.basename(path[:-1]) for path in gcs_w_token.ls('pangeo-data/gross/ws_mask/amazonas')]
    #ds_mask = xr.open_zarr(gcsfs.GCSMap('pangeo-data/gross/ws_mask/amazonas'))
else:
    labels = os.listdir('ws_mask/amazonas')
    #ds_mask = xr.open_zarr('ws_mask/amazonas')
#labels = list(ds_mask.label.values)
len(labels)

In [ ]:
labels_without_vs = list(labels)
labels_with_vs = []
for label in tqdm(labels):
    if is_pangeo_data:
        ds = xr.open_zarr(gcsfs.GCSMap(f'pangeo-data/gross/ws_mask/amazonas/{label}'))
    else:
        ds = xr.open_zarr(f'ws_mask/amazonas/{label}')
    da = ds['mask']
    olat, olon = da.attrs['outlet']
    idx = df_ll[(olat-0.25/1200<df_ll.new_lat.values) & (df_ll.new_lat.values<olat+0.25/1200) & (olon-0.25/1200<df_ll.new_lon.values) & (df_ll.new_lon.values<olon+0.25/1200)].index.values
    labels_with_vs.append(label)
    if len(idx) > 0:
        labels_without_vs.remove(label)

In [ ]:
label_tree = get_label_tree(labels_with_vs)

In [ ]:
d0, d1 = '2000-03-01 12:00:00', '2018-12-31'
path_mask = 'gs:pangeo-data/gross/ws_mask/amazonas'
precipitation = get_precipitation(d0, d1, path_mask, labels)

In [ ]:
if is_pangeo_data:
    gcs = gcsfs.GCSFileSystem(project='pangeo-data', token=gcs_w_token.session.credentials)
    path_mask = 'gs:pangeo-data/gross/ws_mask/amazonas'
    ds_trmm = xr.open_zarr(gcsfs.GCSMap('pangeo-data/trmm_3b42rt', gcs))
    ds_gpm = xr.open_zarr(gcsfs.GCSMap('pangeo-data/gpm_imerg_early', gcs))
    ds_pet = xr.open_zarr(gcsfs.GCSMap('pangeo-data/cgiar_pet', gcs))
else:
    mask_path = 'ws_mask/amazonas'

In [ ]:
for label in labels:
    labels_done = [os.path.basename(path[:-1]) for path in gcs_w_token.ls('pangeo-data/gross/ws_precipitations/amazonas')]
    if label not in labels_done:
        gcs = gcsfs.GCSFileSystem(project='pangeo-data', token=gcs_w_token.session.credentials)
        precipitation = get_precipitation(d0, d1, f'{path_mask}/{label}', ds_trmm, ds_gpm, gcs)
        precipitation.to_pickle('precipitation.pkl')
        gcs_w_token.put('precipitation.pkl', f'pangeo-data/gross/ws_precipitation/amazonas/{label}.pkl')

In [ ]:
for label in labels:
    labels_done = [os.path.basename(path[:-1]) for path in gcs_w_token.ls('pangeo-data/gross/ws_pet/amazonas')]
    if label not in labels_done:
        gcs = gcsfs.GCSFileSystem(project='pangeo-data', token=gcs_w_token.session.credentials)
        pet = get_pet(d0, d1, f'{path_mask}/{label}', ds_pet, gcs)
        pet.to_pickle('pet.pkl')
        gcs_w_token.put('pet.pkl', f'pangeo-data/gross/ws_pet/amazonas/{label}.pkl')

In [ ]:
label = '0'
df = DataFrame()
df['precipitation'] = df_precipitation[label]
df['pet'] = df_pet[label]
df.plot(figsize=(15, 5))